In [1]:
import results_reader.results_reader as rr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv(r"C:\Users\Szymon\OneDrive\praca_inzyneirska\wyniki\results_v2.csv", index_col=0)

In [3]:
colormap = {"MySQL InnoDB": "purple",
          "MySQL MyISAM": "orange",
          "MariaDB InnoDB": "green",
          "MariaDB Aria": "pink",
          "MSSQL": "red",
          "Oracle": "brown",
          "Postgres": "blue"}

legend = [("MySQL", "InnoDB", "purple"),
          ("MySQL", "MyISAM", "orange"),
          ("MariaDB", "InnoDB", "green"),
          ("MariaDB", "Aria", "pink"),
          ("MSSQL", "MSSQL", "red"),
          ("Oracle", "Oracle", "brown"),
          ("Postgres", "Postgres", "blue")]

markevery=np.array([[10000], [100000], [1000000]])

In [4]:
# mysql_innodb_path = r"C:\Users\szymon\OneDrive\praca_inzyneirska\wyniki\mysql_innodb"
# mysql_myisam_path = r"C:\Users\szymon\OneDrive\praca_inzyneirska\wyniki\mysql_myisam"
# mariadb_innodb_path = r"C:\Users\szymon\OneDrive\praca_inzyneirska\wyniki\mariadb_innodb"
# mariadb_aria_path = r"C:\Users\szymon\OneDrive\praca_inzyneirska\wyniki\mariadb_aria"
# mssql_path = r"C:\Users\szymon\OneDrive\praca_inzyneirska\wyniki\mssql"
# postgres_path = r"C:\Users\Szymon\OneDrive\praca_inzyneirska\wyniki\postgres"
# oracle_path = r"C:\Users\Szymon\OneDrive\praca_inzyneirska\wyniki\oracle"
# cpu_cores_mapper = rr.CPUCoresMapper({"results_1cpus": 1,
#                                    "results_2cpus": 2,
#                                    "results_4cpus": 4})
# indexing_mapper = rr.IndexingMapper({"before_index": False,
#                                   "after_index": True})

In [5]:
# df = rr.DatabaseResultsReader(mysql_innodb_path, "MySQL", "InnoDB", cpu_cores_mapper, indexing_mapper).getDataFrame(2)
# df = df.append(rr.DatabaseResultsReader(mysql_myisam_path, "MySQL", "MyISAM", cpu_cores_mapper, indexing_mapper).getDataFrame(2), ignore_index=True)
# df = df.append(rr.DatabaseResultsReader(mariadb_innodb_path, "MariaDB", "InnoDB", cpu_cores_mapper, indexing_mapper).getDataFrame(2), ignore_index=True)
# df = df.append(rr.DatabaseResultsReader(mssql_path, "MSSQL", "MSSQL", cpu_cores_mapper, indexing_mapper).getDataFrame(2), ignore_index=True)
# df = df.append(rr.DatabaseResultsReader(postgres_path, "Postgres", "Postgres", cpu_cores_mapper, indexing_mapper).getDataFrame(2), ignore_index=True)
# df = df.append(rr.DatabaseResultsReader(mariadb_aria_path, "MariaDB", "Aria", cpu_cores_mapper, indexing_mapper).getDataFrame(2), ignore_index=True)
# df = df.append(rr.DatabaseResultsReader(oracle_path, "Oracle", "Oracle", cpu_cores_mapper, indexing_mapper).getDataFrame(2), ignore_index=True)

In [6]:
df[(df['test_name'] == "group_by_order") & (df['query_type'] == "ALL") & (df['table_size'] == 10000000)]

,database,engine,cpus,indexed_table,query,query_type,real,real_stddev,result_size,sys,sys_stddev,table_size,test_name,test_type,user,user_stddev,was_indexed
10,MySQL,InnoDB,1.0,clients_base_all,clients_base_all,ALL,3.7383,0.0132,923828.0,0.0281,0.0109,10000000.0,group_by_order,select,0.1969,0.0109,1.0
18,MySQL,InnoDB,1.0,clients_base_all,clients_base_all,ALL,19.9600,0.0831,923828.0,0.0348,0.0104,10000000.0,group_by_order,select,0.1995,0.0115,0.0
26,MySQL,InnoDB,2.0,clients_base_all,clients_base_all,ALL,3.6195,0.0110,923828.0,0.0428,0.0050,10000000.0,group_by_order,select,0.1854,0.0050,1.0
34,MySQL,InnoDB,2.0,clients_base_all,clients_base_all,ALL,17.4298,0.1089,923828.0,0.0401,0.0154,10000000.0,group_by_order,select,0.1904,0.0159,0.0
42,MySQL,InnoDB,4.0,clients_base_all,clients_base_all,ALL,3.6212,0.0123,923828.0,0.0305,0.0102,10000000.0,group_by_order,select,0.1938,0.0097,1.0
50,MySQL,InnoDB,4.0,clients_base_all,clients_base_all,ALL,17.4829,0.0546,923828.0,0.0368,0.0044,10000000.0,group_by_order,select,0.1941,0.0046,0.0
380,MySQL,MyISAM,1.0,clients_base_all,clients_base_all,ALL,4.3717,0.0114,923828.0,0.0400,0.0108,10000000.0,group_by_order,select,0.1995,0.0099,1.0
381,MySQL,MyISAM,1.0,clients_base_all,clients_base_all,ALL,26.3561,0.3080,923828.0,0.0406,0.0084,10000000.0,group_by_order,select,0.2011,0.0096,0.0
396,MySQL,MyISAM,2.0,clients_base_all,clients_base_all,ALL,4.2517,0.0199,923828.0,0.0424,0.0109,10000000.0,group_by_order,select,0.1945,0.0114,1.0
397,MySQL,MyISAM,2.0,clients_base_all,clients_base_all,ALL,24.3059,0.2436,923828.0,0.0395,0.0078,10000000.0,group_by_order,select,0.2034,0.0084,0.0


In [7]:
df.describe()

,cpus,real,real_stddev,result_size,sys,sys_stddev,table_size,user,user_stddev,was_indexed
count,2519.000000,2519.000000,2519.000000,2.330000e+03,2519.000000,2519.000000,1.184000e+03,2519.000000,2519.000000,2162.000000
mean,2.338626,11.083829,0.296020,1.672261e+06,0.599461,0.026054,1.388889e+06,4.199487,0.071824,0.462997
std,1.249011,30.006683,1.346008,3.643231e+06,3.004700,0.075640,3.272047e+06,15.554176,0.230571,0.498744
min,1.000000,0.005000,0.000300,1.000000e+00,0.000000,0.000900,1.000000e+00,0.000900,0.000900,0.000000
25%,1.000000,0.017650,0.001000,1.000000e+00,0.003500,0.002500,7.750000e+01,0.004000,0.002600,0.000000
50%,2.000000,0.042700,0.007300,1.000000e+02,0.007900,0.003500,5.500000e+03,0.010600,0.003500,0.000000
75%,4.000000,2.463550,0.040300,1.000000e+05,0.041950,0.010200,3.250000e+05,0.187950,0.011800,1.000000
max,4.000000,402.229900,34.168000,1.000000e+07,55.641900,1.836500,1.000000e+07,253.933400,4.799200,1.000000


In [32]:
def plot_results_scatter(df_results, 
                         test_name, 
                         query_type=None, 
                         path=None, 
                         yMin=0,
                         yMax=0,
                         xMin=1,
                         xMax=10000000,
                         indexed=-1,
                         interpolate=True,
                         interpolation_interval=100,
                         no_interpolate_dbs=[]):
    cpus_idx={0: 1, 1: 2, 2: 4}
    df = df_results[(df_results['test_name'] == test_name) & (df_results['query_type'] == query_type.upper())]
    df = df[(df['was_indexed'] == indexed)]
    df = df[['database', 'engine', 'real', 'table_size', 'was_indexed', 'cpus']].reset_index()
    
    width = 25
    height = 25
    fig, ax = plt.subplots(3, figsize=(width, height))
    for idx, cpu in cpus_idx.items():
        for database, engine, color in legend:
            if database != engine:
                label = f"{database} {engine}"
            else:
                label = database
            plot_df = df[(df['database'] == database) & (df['engine'] == engine) & (df['cpus'] == (cpu))]
            plot_df = plot_df[['table_size', 'real']]
#             plot_df = plot_df[(plot_df["table_size"] >= xMin) & plot_df["table_size"] <= xMax]
            plot_df['mark'] = True
            if label == 'Postgres' and cpu == 4:
                print(plot_df)
            if interpolate and label not in no_interpolate_dbs:
                df_append = (pd.DataFrame([val for val in (np.arange((interpolation_interval if xMin < interpolation_interval else xMin), xMax, interpolation_interval)) if val not in [100, 1000, 10000, 100000, 1000000]], columns=['table_size']))
                df_append['mark'] = False
                plot_df = plot_df.append(df_append)
                plot_df = plot_df.sort_values('table_size')
                plot_df = plot_df.interpolate(kind="linear").reset_index()
            else:
                plot_df = plot_df.sort_values('table_size')
            ax[idx].plot(plot_df['table_size'], plot_df['real'], label=label, color=color, marker='o', markevery=plot_df['mark'])
        yMax = df['real'].max() if yMax == 0 else yMax

        ax[idx].set_ylim([yMin, yMax])
        ax[idx].set_ylabel('Czas [s]')
        ax[idx].yaxis.grid(color='grey', linestyle='--')

        ax[idx].set_xscale("log")
        ax[idx].set_xlim([10**0, 10**7])
        ax[idx].set_xlabel('Rozmiar tabeli')
        ax[idx].xaxis.grid(color='grey', linestyle='--')

        ax[idx].set_title(f"{cpu} CPU", fontsize=20)
        ax[idx].legend(fontsize=20, loc='upper left')
        
        for item in ([ax[idx].title, ax[idx].xaxis.label, ax[idx].yaxis.label] +
             ax[idx].get_xticklabels() + ax[idx].get_yticklabels()):
            item.set_fontsize(20)
            
    fig.tight_layout() 

    if path:        
        fig.savefig(path, bbox_inches='tight')
        plt.close()
    else:
        plt.show()

In [36]:
test_type = rr.TestName.GROUP_BY_ORDER.value
query_type = rr.QueryType.ALL.value
was_indexed = 1
yMax = 50
no_interpolate_dbs=['Oracle']
path=rf"C:\Users\Szymon\OneDrive\praca_inzyneirska\wykresy\{test_type}_{query_type}_{was_indexed}.png"

path=path
interpolate=True

plot_results_scatter(df, 
                     test_type, 
                     query_type, 
                     yMax=yMax,
                     indexed=was_indexed,
                     path=path,
                     interpolate=interpolate,
                     xMax=10**7,
                     no_interpolate_dbs=no_interpolate_dbs)

     table_size    real  mark
112  10000000.0  7.2996  True
113         1.0  0.0259  True
114        10.0  0.0263  True
115       100.0  0.0267  True
116      1000.0  0.0285  True
117     10000.0  0.0413  True
118    100000.0  0.1709  True
119   1000000.0  1.4641  True


In [37]:
def create_entry_name(database, engine):
    if database != engine:
        return f"{database} {engine}"
    else:
        return database

In [71]:
test_df = df[(df['test_name'] == 'log') & (df['query_type'] == 'ALL')]
test_df = test_df[(test_df['was_indexed'] == 0) & (test_df['database'] == 'Postgres') & (test_df['engine'] == 'Postgres')]
test_df = test_df[['database', 'engine', 'cpus', 'real', 'table_size']]
test_df['entry'] = test_df.apply(lambda row: create_entry_name(row['database'], row['engine']), axis=1)
test_df = test_df.drop(['database', 'engine'], axis=1)

In [81]:
test_df.pivot(columns='entry', values=['table_size', 'cpus', 'real']).to_csv("test.csv", index=False)